In [ ]:
import os
from dotenv import load_dotenv

from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI as Chat
from langchain.schema import StrOutputParser
from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_text_splitters import CharacterTextSplitter

load_dotenv()


In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [ ]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

print(llm)

In [ ]:
embeddings = OpenAIEmbeddings()

print(embeddings)

In [ ]:
# Initialize Neo4j graph connection
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Initialize Neo4j vector store connection
vector_store = Neo4jVector(
    embeddings,
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print(graph)
print()
print(vector_store)

In [9]:
# Assuming you have a method to retrieve documents from the graph
# Replace with the actual query to retrieve documents
documents = graph.query("MATCH (n) RETURN n LIMIT 25;")

# Convert the retrieved documents into Document objects
document_objects = [Document(page_content=str(doc)) for doc in documents]

db = Neo4jVector.from_documents(
    document_objects, embeddings, url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

print(db)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding) SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS "


In [13]:
stack_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=db.as_retriever()
)

print(stack_retriever)

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11706c080>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11706dca0>, root_client=<openai.OpenAI object at 0x116e6afc0>, root_async_client=<openai.AsyncOpenA

In [19]:
response = stack_retriever.invoke("what is in the db?")

In [20]:
print(response["result"])

The database contains questions and discussions related to technical issues and programming, specifically focusing on topics like Neo4j, encryption in Node.js, and connection strings for databases. The entries include questions about migrating data from RDBMS to Neo4j, specifying database names in connection strings for Neo4j, handling deprecated encryption methods in Node.js, and storing credentials in connection strings for Neo4j.


In [21]:
response

{'query': 'what is in the db?',
 'result': 'The database contains questions and discussions related to technical issues and programming, specifically focusing on topics like Neo4j, encryption in Node.js, and connection strings for databases. The entries include questions about migrating data from RDBMS to Neo4j, specifying database names in connection strings for Neo4j, handling deprecated encryption methods in Node.js, and storing credentials in connection strings for Neo4j.'}